# © Artur Czarnecki. All rights reserved.
# Intergrax framework – proprietary and confidential.
# Use, modification, or distribution without written permission is prohibited.

## Test D1 — DYNAMIC loop: two iterations via ScriptedPlanSource (attempt=0 → attempt=1)

Goal:
- Verify that `PlanLoopController.run_dynamic(...)` performs multiple iterations when the first executed step does NOT produce `state.runtime_answer`.
- Verify that iteration bumping creates a `ReplanContext` with a stable `attempt` counter, so `ScriptedPlanSource` selects the next plan deterministically:
  - iteration #0 uses `attempt=0`
  - iteration #1 uses `attempt=1`

Scenario:
- EnginePlanner uses `ScriptedPlanSource` with exactly 2 plans:
  1) next_step = "synthesize" (draft-only, should not finalize)
  2) next_step = "finalize"   (must produce final `RuntimeAnswer`)

Assertions:
- Returned answer is not empty.
- Debug trace shows two planning iterations and the second iteration runs with `replan_attempt=1`.


In [ ]:
import sys, os

from intergrax.runtime.drop_in_knowledge_mode.planning.engine_plan_models import EngineNextStep, PlanIntent
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..", "..")))

from intergrax.llm_adapters.llm_provider import LLMProvider
from intergrax.llm_adapters.llm_provider_registry import LLMAdapterRegistry

from intergrax.runtime.drop_in_knowledge_mode.config import RuntimeConfig
from intergrax.runtime.drop_in_knowledge_mode.engine.runtime_context import RuntimeContext
from intergrax.runtime.drop_in_knowledge_mode.engine.runtime_state import RuntimeState
from intergrax.runtime.drop_in_knowledge_mode.pipelines.contract import RuntimePipeline

from intergrax.runtime.drop_in_knowledge_mode.planning.engine_planner import EnginePlanner
from intergrax.runtime.drop_in_knowledge_mode.planning.plan_loop_controller import PlanLoopController
from intergrax.runtime.drop_in_knowledge_mode.planning.plan_loop_models import PlanLoopPolicy
from intergrax.runtime.drop_in_knowledge_mode.planning.plan_sources import PlanSpec, ScriptedPlanSource

from intergrax.runtime.drop_in_knowledge_mode.planning.step_executor import StepExecutor
from intergrax.runtime.drop_in_knowledge_mode.planning.step_planner import StepPlanner, StepPlannerConfig

from intergrax.runtime.drop_in_knowledge_mode.responses.response_schema import RuntimeRequest
from intergrax.runtime.drop_in_knowledge_mode.runtime_steps.contract import RuntimeStepRunner
from intergrax.runtime.drop_in_knowledge_mode.runtime_steps.setup_steps_tool import SETUP_STEPS

from intergrax.runtime.drop_in_knowledge_mode.session.in_memory_session_storage import InMemorySessionStorage
from intergrax.runtime.drop_in_knowledge_mode.session.session_manager import SessionManager


# ---------------------------------------------------------------------
# 1) Deterministic plans (ScriptedPlanSource): attempt=0 then attempt=1
# ---------------------------------------------------------------------

plan_0_synthesize = PlanSpec(
    version="1",
    intent=PlanIntent.GENERIC,
    next_step=EngineNextStep.SYNTHESIZE,
    reasoning_summary="Produce a draft first (no final answer yet).",
    ask_clarifying_question=False,
    clarifying_question=None,
    use_websearch=False,
    use_user_longterm_memory=False,
    use_rag=False,
    use_tools=False,
)


plan_1_finalize = PlanSpec(
    version="1",
    intent=PlanIntent.GENERIC,
    next_step=EngineNextStep.FINALIZE,
    reasoning_summary="Finalize the answer using accumulated context/draft.",
    ask_clarifying_question=False,
    clarifying_question=None,
    use_websearch=False,
    use_user_longterm_memory=False,
    use_rag=False,
    use_tools=False,
)


scripted_source = ScriptedPlanSource([plan_0_synthesize, plan_1_finalize])


# ---------------------------------------------------------------------
# 2) Runtime config + context + state (full isolation within this cell)
# ---------------------------------------------------------------------

config = RuntimeConfig(
    llm_adapter=LLMAdapterRegistry.create(LLMProvider.OLLAMA),
    enable_rag=False,
    enable_websearch=False,
    tools_mode="off",
)

session_manager = SessionManager(storage=InMemorySessionStorage())

ctx = RuntimeContext.build(
    config=config,
    session_manager=session_manager,
)

request = RuntimeRequest(
    session_id="planloop-dynamic-test-session",
    user_id="planloop-dynamic-test-user",
    message="""
Context:
- We are testing PlanLoopController.run_dynamic in Intergrax.
- The planner uses ScriptedPlanSource to produce deterministic EnginePlans.
Task:
Return a short answer about what was tested.
""".strip(),
    attachments=[],
)

state = RuntimeState(
    context=ctx,
    request=request,
    run_id="planloop-e2e-run",
)

# Capabilities ON — avoid planner capability clamp in tests
state.cap_websearch_available = True
state.cap_tools_available = True
state.cap_rag_available = True
state.cap_user_ltm_available = True

# Setup baseline runtime state (history/instructions/etc.)
await RuntimeStepRunner.execute_pipeline(SETUP_STEPS, state)


# ---------------------------------------------------------------------
# 3) Planning + execution stack (Intergrax components only)
# ---------------------------------------------------------------------

engine_planner = EnginePlanner(
    llm_adapter=config.llm_adapter,
    plan_source=scripted_source,
)

step_planner = StepPlanner(StepPlannerConfig())

registry = RuntimePipeline.build_default_planning_step_registry()
step_executor = StepExecutor(registry=registry)

policy = PlanLoopPolicy(
    max_replans=3,
    max_same_plan_repeats=2,
)

plan_loop = PlanLoopController(
    engine_planner=engine_planner,
    step_planner=step_planner,
    step_executor=step_executor,
    policy=policy,
)


# ---------------------------------------------------------------------
# 4) Execute DYNAMIC loop
# ---------------------------------------------------------------------

answer = await plan_loop.run_dynamic(
    state=state,
    plan_id_prefix="dyn-d1",
    user_message=state.request.message,
)

print("Answer:", (answer.answer or "")[:200])
print("Route.strategy:", answer.route.strategy)
print("Route.extra:", answer.route.extra)

# ---------------------------------------------------------------------
# 5) Assertions: two iterations and attempt bump visibility in trace
# ---------------------------------------------------------------------

events = state.debug_trace.get("events", [])

planner_iters = [
    e for e in events
    if isinstance(e, dict)
    and e.get("component") == "planner"
    and e.get("step") == "plan_loop_dynamic"
    and e.get("message") == "Planning iteration started."
]

print("planner_iterations_count:", len(planner_iters))
for i, e in enumerate(planner_iters):
    print(f"iter[{i}] replan_attempt:", e.get("data", {}).get("replan_attempt"))

assert len(planner_iters) == 2, f"Expected exactly 2 iterations, got {len(planner_iters)}"
assert planner_iters[0].get("data", {}).get("replan_attempt") is None, "First iteration should have no replan_attempt"
assert planner_iters[1].get("data", {}).get("replan_attempt") == 1, "Second iteration should run with replan_attempt=1"
assert answer.answer is not None and answer.answer.strip() != "", "Expected a non-empty final answer"

print("D1 OK")


Answer: The PlanLoopController's run_dynamic method has been tested with deterministic EnginePlans generated by a ScriptedPlanSource in an Integrax context.
Route.strategy: llm_only
Route.extra: {'used_attachments_context': False, 'attachments_chunks': 0}
planner_iterations_count: 2
iter[0] replan_attempt: None
iter[1] replan_attempt: 1
D1 OK


## Test D2 — DYNAMIC loop: CLARIFY → HITL (Needs User Input)

Goal:
- Verify that `PlanLoopController.run_dynamic(...)` returns a HITL (human-in-the-loop) clarifying question when the planner decides to clarify.
- Ensure the stop behavior is deterministic with `ScriptedPlanSource` and does not execute additional steps.

Scenario:
- Scripted plan #0: intent="clarify", next_step="clarify", ask_clarifying_question=true, clarifying_question="..."
- Expected behavior: `run_dynamic` returns a `RuntimeAnswer` that contains the clarifying question and uses HITL route strategy.

Assertions:
- Returned answer equals the scripted `clarifying_question`.
- `answer.route.strategy == "hitl_clarify"`.
- The loop performs exactly 1 iteration.


In [ ]:
import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..", "..")))

from intergrax.llm_adapters.llm_provider import LLMProvider
from intergrax.llm_adapters.llm_provider_registry import LLMAdapterRegistry

from intergrax.runtime.drop_in_knowledge_mode.config import RuntimeConfig
from intergrax.runtime.drop_in_knowledge_mode.engine.runtime_context import RuntimeContext
from intergrax.runtime.drop_in_knowledge_mode.engine.runtime_state import RuntimeState

from intergrax.runtime.drop_in_knowledge_mode.planning.engine_planner import EnginePlanner
from intergrax.runtime.drop_in_knowledge_mode.planning.plan_loop_controller import PlanLoopController
from intergrax.runtime.drop_in_knowledge_mode.planning.plan_loop_models import PlanLoopPolicy
from intergrax.runtime.drop_in_knowledge_mode.planning.plan_sources import PlanSpec, ScriptedPlanSource

from intergrax.runtime.drop_in_knowledge_mode.planning.step_executor import StepExecutor
from intergrax.runtime.drop_in_knowledge_mode.planning.step_planner import StepPlanner, StepPlannerConfig

from intergrax.runtime.drop_in_knowledge_mode.responses.response_schema import RuntimeRequest
from intergrax.runtime.drop_in_knowledge_mode.runtime_steps.contract import RuntimeStepRunner
from intergrax.runtime.drop_in_knowledge_mode.runtime_steps.setup_steps_tool import SETUP_STEPS

from intergrax.runtime.drop_in_knowledge_mode.pipelines.contract import RuntimePipeline
from intergrax.runtime.drop_in_knowledge_mode.session.in_memory_session_storage import InMemorySessionStorage
from intergrax.runtime.drop_in_knowledge_mode.session.session_manager import SessionManager


# ---------------------------------------------------------------------
# 1) Scripted plan: CLARIFY
# ---------------------------------------------------------------------

CLAR_Q = "Could you clarify which part of the DYNAMIC loop you want to validate: replanning, limits, or HITL behavior?"

plan_clarify = PlanSpec(
    version="1",
    intent=PlanIntent.CLARIFY,
    next_step=EngineNextStep.CLARIFY,
    reasoning_summary="Need user clarification before proceeding.",
    ask_clarifying_question=True,
    clarifying_question=CLAR_Q,
    use_websearch=False,
    use_user_longterm_memory=False,
    use_rag=False,
    use_tools=False,
)


scripted_source = ScriptedPlanSource([plan_clarify])


# ---------------------------------------------------------------------
# 2) Runtime config + context + state (full isolation within this cell)
# ---------------------------------------------------------------------

config = RuntimeConfig(
    llm_adapter=LLMAdapterRegistry.create(LLMProvider.OLLAMA),
    enable_rag=False,
    enable_websearch=False,
    tools_mode="off",
)

session_manager = SessionManager(storage=InMemorySessionStorage())

ctx = RuntimeContext.build(
    config=config,
    session_manager=session_manager,
)

request = RuntimeRequest(
    session_id="planloop-dynamic-test-session-d2",
    user_id="planloop-dynamic-test-user",
    message="Test D2: Planner should request clarification.",
    attachments=[],
)

state = RuntimeState(
    context=ctx,
    request=request,
    run_id="planloop-e2e-run",
)

# Capabilities ON — avoid planner capability clamp in tests
state.cap_websearch_available = True
state.cap_tools_available = True
state.cap_rag_available = True
state.cap_user_ltm_available = True

# Setup baseline runtime state
await RuntimeStepRunner.execute_pipeline(SETUP_STEPS, state)


# ---------------------------------------------------------------------
# 3) Planning + execution stack (Intergrax components only)
# ---------------------------------------------------------------------

engine_planner = EnginePlanner(
    llm_adapter=config.llm_adapter,
    plan_source=scripted_source,
)

step_planner = StepPlanner(StepPlannerConfig())

registry = RuntimePipeline.build_default_planning_step_registry()
step_executor = StepExecutor(registry=registry)

policy = PlanLoopPolicy(
    max_replans=3,
    max_same_plan_repeats=2,
)

plan_loop = PlanLoopController(
    engine_planner=engine_planner,
    step_planner=step_planner,
    step_executor=step_executor,
    policy=policy,
)


# ---------------------------------------------------------------------
# 4) Execute DYNAMIC loop
# ---------------------------------------------------------------------

answer = await plan_loop.run_dynamic(
    state=state,
    plan_id_prefix="dyn-d2",
    user_message=state.request.message,
)

print("Answer:", answer.answer)
print("Route.strategy:", answer.route.strategy)
print("Route.extra:", answer.route.extra)


# ---------------------------------------------------------------------
# 5) Assertions
# ---------------------------------------------------------------------

events = state.debug_trace.get("events", [])

planner_iters = [
    e for e in events
    if isinstance(e, dict)
    and e.get("component") == "planner"
    and e.get("step") == "plan_loop_dynamic"
    and e.get("message") == "Planning iteration started."
]

print("planner_iterations_count:", len(planner_iters))

assert answer.answer == CLAR_Q, "Expected the runtime answer to be exactly the clarifying question."
assert answer.route.strategy == "hitl_clarify", f"Expected route.strategy='hitl_clarify', got {answer.route.strategy!r}"
assert len(planner_iters) == 1, f"Expected exactly 1 iteration, got {len(planner_iters)}"

print("D2 OK")


Answer: Could you clarify which part of the DYNAMIC loop you want to validate: replanning, limits, or HITL behavior?
Route.strategy: hitl_clarify
Route.extra: {'stop_reason': 'needs_user_input', 'origin_step_id': <StepId.CLARIFY: 'clarify'>, 'context_key': 'clarify.user_input', 'must_answer_to_continue': True}
planner_iterations_count: 1
D2 OK


## Test D3 — DYNAMIC loop: max iterations enforced (hard fail)

Goal:
- Verify that `PlanLoopController.run_dynamic(...)` enforces the iteration budget when it cannot produce a final answer.
- Ensure the loop does NOT run indefinitely; it must stop with a clear diagnostic error.

Scenario:
- ScriptedPlanSource provides multiple plans that never finalize:
  - next_step = "synthesize" (no finalize; no explicit replan request)
- Policy: max_replans=2 (used as iteration budget).
- Expected behavior:
  - The loop runs for a limited number of iterations and then raises `RuntimeError`
    with message "max iterations exceeded".

Assertions:
- A RuntimeError is raised.
- Error message contains "max iterations exceeded".
- Planner iterations count > 1 (it actually looped before stopping).


In [ ]:
import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..", "..")))

from intergrax.llm_adapters.llm_provider import LLMProvider
from intergrax.llm_adapters.llm_provider_registry import LLMAdapterRegistry

from intergrax.runtime.drop_in_knowledge_mode.config import RuntimeConfig
from intergrax.runtime.drop_in_knowledge_mode.engine.runtime_context import RuntimeContext
from intergrax.runtime.drop_in_knowledge_mode.engine.runtime_state import RuntimeState

from intergrax.runtime.drop_in_knowledge_mode.planning.engine_planner import EnginePlanner
from intergrax.runtime.drop_in_knowledge_mode.planning.plan_loop_controller import PlanLoopController
from intergrax.runtime.drop_in_knowledge_mode.planning.plan_loop_models import PlanLoopPolicy
from intergrax.runtime.drop_in_knowledge_mode.planning.plan_sources import PlanSpec, ScriptedPlanSource

from intergrax.runtime.drop_in_knowledge_mode.planning.step_executor import StepExecutor
from intergrax.runtime.drop_in_knowledge_mode.planning.step_planner import StepPlanner, StepPlannerConfig

from intergrax.runtime.drop_in_knowledge_mode.responses.response_schema import RuntimeRequest
from intergrax.runtime.drop_in_knowledge_mode.runtime_steps.contract import RuntimeStepRunner
from intergrax.runtime.drop_in_knowledge_mode.runtime_steps.setup_steps_tool import SETUP_STEPS

from intergrax.runtime.drop_in_knowledge_mode.pipelines.contract import RuntimePipeline
from intergrax.runtime.drop_in_knowledge_mode.session.in_memory_session_storage import InMemorySessionStorage
from intergrax.runtime.drop_in_knowledge_mode.session.session_manager import SessionManager


# ---------------------------------------------------------------------
# 1) Scripted plans: never finalize (force max-iterations escalation)
# ---------------------------------------------------------------------

def mk_synthesize_plan(i: int) -> PlanSpec:
    return PlanSpec(
        version="1",
        intent=PlanIntent.GENERIC,
        next_step=EngineNextStep.SYNTHESIZE,
        reasoning_summary=f"Iteration {i}: keep synthesizing without finalizing.",
        ask_clarifying_question=False,
        clarifying_question=None,
        use_websearch=False,
        use_user_longterm_memory=False,
        use_rag=False,
        use_tools=False,
    )


# Provide more plans than max_replans budget so the loop must escalate.
scripted_source = ScriptedPlanSource([mk_synthesize_plan(0), mk_synthesize_plan(1), mk_synthesize_plan(2), mk_synthesize_plan(3)])


# ---------------------------------------------------------------------
# 2) Runtime config + context + state (full isolation within this cell)
# ---------------------------------------------------------------------

config = RuntimeConfig(
    llm_adapter=LLMAdapterRegistry.create(LLMProvider.OLLAMA),
    enable_rag=False,
    enable_websearch=False,
    tools_mode="off",
)

session_manager = SessionManager(storage=InMemorySessionStorage())

ctx = RuntimeContext.build(
    config=config,
    session_manager=session_manager,
)

request = RuntimeRequest(
    session_id="planloop-dynamic-test-session-d3",
    user_id="planloop-dynamic-test-user",
    message="Test D3: DYNAMIC loop should escalate when it cannot finalize within limits.",
    attachments=[],
)

state = RuntimeState(
    context=ctx,
    request=request,
    run_id="planloop-e2e-run",
)

# Capabilities ON — avoid planner capability clamp in tests
state.cap_websearch_available = True
state.cap_tools_available = True
state.cap_rag_available = True
state.cap_user_ltm_available = True

# Setup baseline runtime state
await RuntimeStepRunner.execute_pipeline(SETUP_STEPS, state)


# ---------------------------------------------------------------------
# 3) Planning + execution stack (Intergrax components only)
# ---------------------------------------------------------------------

engine_planner = EnginePlanner(
    llm_adapter=config.llm_adapter,
    plan_source=scripted_source,
)

step_planner = StepPlanner(StepPlannerConfig())

registry = RuntimePipeline.build_default_planning_step_registry()
step_executor = StepExecutor(registry=registry)

# IMPORTANT: force HITL behavior on limit
policy = PlanLoopPolicy(
    max_replans=2,
    max_same_plan_repeats=999,   # do not trip same-plan guard; we want max-iterations guard
    on_max_replans="hitl",       # expects your policy supports this; if not, default should still HITL in your run_dynamic
)

plan_loop = PlanLoopController(
    engine_planner=engine_planner,
    step_planner=step_planner,
    step_executor=step_executor,
    policy=policy,
)


# ---------------------------------------------------------------------
# 4) Execute DYNAMIC loop
# ---------------------------------------------------------------------

events = state.debug_trace.get("events", [])

try:
    _ = await plan_loop.run_dynamic(
        state=state,
        plan_id_prefix="dyn-d3",
        user_message=state.request.message,
    )
    raise AssertionError("Expected RuntimeError due to max iterations exceeded, but run_dynamic returned normally.")
except RuntimeError as e:
    msg = str(e)
    print("Caught RuntimeError:", msg)
    assert "max iterations exceeded" in msg, "Expected error message to contain 'max iterations exceeded'."

planner_iters = [
    e for e in events
    if isinstance(e, dict)
    and e.get("component") == "planner"
    and e.get("step") == "plan_loop_dynamic"
    and e.get("message") == "Planning iteration started."
]

print("planner_iterations_count:", len(planner_iters))
assert len(planner_iters) > 1, "Expected more than 1 iteration before stopping."

print("D3 OK")


Caught RuntimeError: PlanLoopController(DYNAMIC): max iterations exceeded. max_replans=2 iterations_used=3
planner_iterations_count: 3
D3 OK
